# 规则验证模块

## 主要任务

规则验证模块的主要任务，是将硬匹配和软匹配的结果进行二次打标来验证。因为规则的标注并非完全准确，原本需要人去验证，但是人力验证成本过高，因此使用模型来代替人进行验证。

这个模块需要一些训练语料来训练模型，同时需要上一轮的软硬匹配结果来进行二次打标。

## 具体流程

1. 模型训练
2. 模型评估
3. 模型预测 / 句子打标
4. 冲突检验


# 实验数据准备

将 sent_train 数据集中的句子和标签全部利用来训练模型

利用 Word2vec 将句子变成向量

In [4]:
# 导入实验数据
import pandas as pd

train_data = pd.read_csv("/Users/xuhaoshuai/GitHub/HumanIE_IPM/data/sent_train/brother.csv")
verify_data = pd.read_csv("/Users/xuhaoshuai/GitHub/HumanIE_IPM/data/sent_veri/brother.csv")

# 抽取句子部分
train_sent_set = train_data['sent']
verify_sent_set = verify_data['sent']

# 将 DataFrame 变成列表
def split_by_space(sentence_set):
    sent_word_list = []
    for sentence in sentence_set:
        sent_word_list.append(sentence.split())
    return sent_word_list

# train_sent_word_list = []
# for sent in train_sent_set.tolist():
#     train_sent_word_list.append(sent.split())

train_sent_word_list = split_by_space(train_sent_set)
verify_sent_word_list = split_by_space(verify_sent_set)


In [5]:
# # 加载和调用 Word2vec，构造模型的训练向量
import numpy as np
from gensim.models import Word2Vec

model = Word2Vec.load("/Users/xuhaoshuai/GitHub/HumanIE_IPM/src/softmatch/word2vec.model")

def sent2vec(sent_word_list):
    X = []
    for sent_word in train_sent_word_list:
        sent_vec = np.zeros(100)
        word_vec_sum = np.zeros(100)
        for word in sent_word:
            word_vec_sum = word_vec_sum + model.wv[word]
        sent_vec = word_vec_sum / len(sent_word)
        X.append(sent_vec.tolist())
    return X

X_train = sent2vec(train_sent_word_list)
X_verify = sent2vec(verify_sent_word_list)

y_train = np.array(train_data['label'].tolist())
y_verify = np.array(train_data['label'].tolist())

In [6]:
# 分割数据为训练集和测试集
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import f1_score

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
# 模型评估
hard_svm = svm.SVC()
soft_svm = svm.SVC()
hard_svm.fit(X_train, y_train)
soft_svm.fit(X_train, y_train)

y_pred_hard = hard_svm.predict(X_verify)
y_pred_soft = soft_svm.predict(X_verify)

hard_f1 = f1_score(y_verify, y_pred_hard, average='binary')
soft_f1 = f1_score(y_verify, y_pred_soft, average='binary')

0.7723502304147465


# 模型预测
pred_data = pd.read_

In [ ]:
# 模型预测
